In [1]:
import os
import glob
import numpy as np
from itkwidgets import view

In [2]:
def normalize(x):
    M = 240
    m = -1024
    x = (x-m)/(M-m)
    x = 2*x -1
    return x

In [3]:
def unnormalize(x):
    M = 240
    m = -1024
    x = 0.5*(x+1)*(M-m) + m
    return x

In [4]:
import torch
import torch.nn as nn
from models.networks3d_resnet import ResnetGenerator3d as PatchGenerator

In [5]:
model_A = PatchGenerator(1, 1, n_blocks=9)
model_B = PatchGenerator(1, 1, n_blocks=9)

In [6]:
# lung segmentation file dictionary
lung_region_dir = '/ocean/projects/asc170022p/lisun/copd/gnn_shared/data/patch_data_32_6_reg_mask/lung_region'
lung_region_files = glob.glob(lung_region_dir + '/*.npy')
lung_region_dict = {}
for f in lung_region_files:
    s = f.split('/')
    t = s[-1]
    k = t[:6]
    lung_region_dict[k] = f

In [25]:
# Atlas locations
altas_loc_file = '/ocean/projects/asc170022p/lisun/copd/gnn_shared/data/patch_data_32_6_reg_mask/19676E_INSP_STD_JHU_COPD_BSpline_Iso1_patch_loc.npy'
atlas_loc = np.load(altas_loc_file)
atlas_loc = (atlas_loc / atlas_loc.max(0))*2 -1



## Low to High

In [8]:
PATH_A = '/ocean/projects/asc170022p/rohit33/cyclegan_models/checkpoints/copd_emphysema_resnet9/iter_420000_net_G_A.pth'
model_A.load_state_dict(torch.load(PATH_A))
model_A.eval()

ResnetGenerator3d(
  (embed): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=256, out_features=256, bias=True)
  )
  (converter): Conv3d(512, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (encoder): Sequential(
    (0): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3))
    (1): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (4): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (7): InstanceNor

In [9]:
PATH_B = '/ocean/projects/asc170022p/rohit33/cyclegan_models/checkpoints/copd_emphysema_resnet9/iter_420000_net_G_B.pth'
model_B.load_state_dict(torch.load(PATH_B))
model_B.eval()

ResnetGenerator3d(
  (embed): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=256, out_features=256, bias=True)
  )
  (converter): Conv3d(512, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (encoder): Sequential(
    (0): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3))
    (1): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (4): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (7): InstanceNor

In [192]:
patch_idx = 120
#patch_dir = os.path.join('/ocean/projects/asc170022p/rohit33/emphysemapatches/lo/', str(patch_idx))
patch_dir = os.path.join('/ocean/projects/asc170022p/rohit33/emphysemapatches/hi/', str(patch_idx))
patch_files = glob.glob(patch_dir + '/*.npy')

In [193]:
patch_files[:5]

['/ocean/projects/asc170022p/rohit33/emphysemapatches/hi/120/12220E.npy',
 '/ocean/projects/asc170022p/rohit33/emphysemapatches/hi/120/17476M.npy',
 '/ocean/projects/asc170022p/rohit33/emphysemapatches/hi/120/22931R.npy',
 '/ocean/projects/asc170022p/rohit33/emphysemapatches/hi/120/17858A.npy',
 '/ocean/projects/asc170022p/rohit33/emphysemapatches/hi/120/23618S.npy']

In [194]:
# extract pid
pid_lst = []
for f in patch_files:
    s = f.split('.')
    pid_lst.append(s[0][-6:])

In [195]:
f_idx = 1
pid = pid_lst[f_idx]
patch_arr = np.load(patch_files[f_idx])

In [196]:
# real image A
view(patch_arr, axes=True, gradient_opacity=0.9)

Viewer(axes=True, geometries=[], gradient_opacity=0.9, point_sets=[], rendered_image=<itk.itkImagePython.itkIm…

In [197]:
# lung segmentation
lung_region = np.load(lung_region_dict[pid])[patch_idx].squeeze()
lung_region_mask = lung_region > 0
lung_region_mask = lung_region_mask.astype(int)

In [198]:
view(lung_region_mask, axes=True, gradient_opacity=0.1)

Viewer(axes=True, geometries=[], gradient_opacity=0.1, point_sets=[], rendered_image=<itk.itkImagePython.itkIm…

In [199]:
# real emphysema mask
emph_region_mask_A = (patch_arr < -950) & (lung_region_mask > 0)
emph_region_mask_A = emph_region_mask_A.astype(int)

In [200]:
emph_region_mask_A.sum()

19485

In [201]:
view(emph_region_mask_A, axes=True, gradient_opacity=0.1)

Viewer(axes=True, geometries=[], gradient_opacity=0.1, point_sets=[], rendered_image=<itk.itkImagePython.itkIm…

In [202]:
# fake image B
patch_arr_norm = normalize(patch_arr)
patch_tensor = torch.from_numpy(patch_arr_norm).unsqueeze(0).unsqueeze(0) # B=1, C=1, W=32, D=32, H=32
patchloc = torch.from_numpy(atlas_loc[patch_idx, :]).unsqueeze(0).float() # B=1, 3
fake_B_tensor = model_B(patch_tensor, patchloc)
fake_B_arr = fake_B_tensor.squeeze(0).squeeze(0).detach().numpy()
fake_B_arr_unnorm = unnormalize(fake_B_arr)

In [203]:
view(fake_B_arr_unnorm, axes=True, gradient_opacity=0.9)

Viewer(axes=True, geometries=[], gradient_opacity=0.9, point_sets=[], rendered_image=<itk.itkImagePython.itkIm…

In [204]:
# real emphysema mask
emph_region_mask_B = (fake_B_arr_unnorm < -950) & (lung_region_mask > 0)
emph_region_mask_B = emph_region_mask_B.astype(int)

In [205]:
emph_region_mask_B.sum()

30999